**[An Automatic Procedure for the Quantitative Characterization of Submarine Bedforms](http://www.mdpi.com/2076-3263/8/1/28/htm)**


**Abstract**


A model for the extraction and quantitative characterization of submarine landforms from high-resolution digital bathymetry is presented. The procedure is fully automated and comprises two parts. The first part consists of an analytical model which extracts quantitative information from a Digital Elevation Model in the form of objects with similar parametric characteristics (terrain objects). The second part is a rule-based model where the terrain objects are reclassified into distinct landforms with well-defined three dimensional characteristics. For the focus of this work, the quantitative characterization of isolated dunes (height greater than 2 m) is used to exemplify the process.
The primary metrics used to extract terrain objects are the flatness threshold and the search radius, which are then used by the analytical model to identify the feature type. Once identified as dunes, a sequence of spatial analysis routines is applied to identify and compute metrics for each dune including length, height, width, ray of curvature, slope analysis for each stoss and lee side, and dune symmetry. 
Dividing the model into two parts, one scale-dependent and another centered around the shape of the landform, makes the model applicable to other submarine landforms like ripples, mega-ripples, and coral reefs, which also have well-defined three-dimensional characteristics.

In [ ]:
!g.remove -f type=raster name=bathy_2015_el

In [ ]:
!unzip grassdata.zip

In [ ]:
!mv home/epinux/grassdata/ /home/epinux

**Keywords:** geomorphometry; GIS; spatial scale; spatial analysis; terrain analysis;

* **GRASS GIS (Version 7.2) with the following `GRASS GIS ADDONS` installed via [`g.extension`](https://grass.osgeo.org/grass72/manuals/g.extension):**
    * [`r.geomorphon`](https://grass.osgeo.org/grass74/manuals/addons/r.geomorphon.html)
    * [`r.area`](https://grass.osgeo.org/grass74/manuals/addons/r.area.html)
    * [`v.transects`](https://grass.osgeo.org/grass74/manuals/addons/v.transects.html)

# Code import

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
import datetime as dt
start=dt.datetime.now()

In [ ]:
import numpy as np

In [ ]:
import grass.script as grass

In [ ]:
from IPython.core.display import display

In [ ]:
from ipygrass.grender import makefigure, makemap, makelegend
from ipygrass.gisutils import grouper, list2table, list2dict, getcoords, getprofile
from ipygrass.grassutil import General as g
from ipygrass.grassutil import Raster as r

* **Get the bathymetry data**

In [ ]:
url = 'https://ndownloader.figshare.com/files/15008201'
import urllib.request
urllib.request.urlretrieve(url, filename='tmp/bathy_2015.tif')

* **Import the data**

    * `GRASS GIS` command: [`r.in.gdal`](https://grass.osgeo.org/grass74/manuals/r.in.gdal)

In [ ]:
g.run_command('r.in.gdal', 
                  input='tmp/bathy_2015.tif', 
                  output='bathy_2015_el', 
                  flags='oe', 
                  overwrite=True)

In [ ]:
g.read_command('g.region', 
                   raster='bathy_2015_el',
                   flags='ap', htable=True, sep=':')

* **Set the vertical reference to the Geoid** ($-34.42 \ m$)

    * `GRASS GIS` command: [`r.mapcalc`](https://grass.osgeo.org/grass74/manuals/r.mapcalc) 

In [ ]:
g.run_command('r.mapcalc' , 
                  expression = '%s = %s+abs(%s)' % ('bathy_2015', 
                                                    'bathy_2015_el', 
                                                    '-34.42'), 
                  overwrite=True)

* **Set the colortable to `haxby` with histogram equalization**

    * `GRASS GIS` command: [`r.colors`](https://grass.osgeo.org/grass74/manuals/r.colors)

In [ ]:
g.run_command('r.colors', color='haxby', map='bathy_2015', flags='e')

In [ ]:
makelegend(raster='bathy_2015', fontsize=8, units=' m', label='Bathy 2015')
makefigure(['bathy_2015'],
           caption=[1, 'MBES 2015, cell size: $1 \\times 1 \\ [m]$'])


In [ ]:
#uncomment to run an ipyleaflet widget# makemap(['bathy_2015'], zoom=15, caption=[1,'$\\text{MBES 2015}$'])

* **Set the `GRASS GIS` working region to the extent of the control unit #1 (visually identified sand dune)**

```
north:      4546380
south:      4546202
west:       506185
east:       506571
```

In [ ]:
g.read_command('g.region', 
                   n=4546380, 
                   s=4546202,
                   w=506185,
                   e=506571,
                   flags='ap', htable=True, sep=':')

In [ ]:
makefigure(['bathy_2015'], 
           caption=[2, 
                    'Control unit #1, cell size: $1 \\times 1 \\ [m]$'])


# GIS Rule based Model-GRM


The whole GRM workflow can be divided into three sections:
1. Extract and vectorize sand wave crest (SWC).
2. Extract and vectorize sand wave main body (SW).
3. Identify lee and stoss side and compute sand wave’s metrics.

The sequence of each step are summarized and visually described below:

## Extract and vectorize sand wave crest for each bedform

###  Extraction of sand wave crest (SWC) 
From the TFE results (r.geomorphon), SWC are identified by extracted by reclassifying the cells with feature type equal to ridge and summit (feature type class: 2,3) into a new raster feature with category value 1 and setting the remaining cells to null (Figure 7).

In [ ]:
g.run_command('r.geomorphon', 
                  elevation='bathy_2015',
                  forms='geomorphometry',
                  search=9,
                  skip=3,
                  flat=2.0,
                  dist=0,
                  step=0,
                  start=0, 
                  overwrite=True)

In [ ]:
g.run_command('r.mapcalc' , 
                  expression = '%s=if(%s==3 | %s==2,1, null())' % ('swc','geomorphometry', 'geomorphometry'), 
                  overwrite=True)

In [ ]:
makefigure(['geomorphometry', 
            'swc'], 
           caption=[7, 'Extraction of sand wave crest (SWC).'])

###  SWC thinning
SWC areas are thinned and reduced to a single pixel width (Figure 8).

In [ ]:
g.run_command('r.thin', 
                  input='swc',
                  output='swc_thin',
                  iterations=400,
                  overwrite=True)

In [ ]:
makefigure(['swc', 
            'swc_thin'], 
           caption=[8, 'SWC thinning.'])

###  SWC clumping
Each SWC is recategorized by grouping cells that form physically discrete areas into unique categories and assign a distinct color to each raster feature, different colors are assigned to each linear feature (Figure 9).

In [ ]:
g.run_command('r.clump',
                  input='swc_thin',
                  output='swc_thin_clump',
                  flags='d',
                  overwrite=True)

In [ ]:
makefigure(['swc_thin', 
            'swc_thin_clump'], 
           caption=[9, 'SWC clumping.'])

###  SWC filtering by length
Each feature with same category shorter than a given threshold is removed (Figure 10).

In [ ]:
g.run_command('r.area', 
                  input='swc_thin_clump', 
                  output='swc_thin_long',
                  lesser=70,
                  overwrite=True)

In [ ]:
makefigure(['swc_thin_clump', 
            'swc_thin_long'], 
           caption=[10, 'SWC filtering by length.'])

###  SWC vectorization 
Conversion from raster to vector to obtain a vector feature of type line representing an approximate sand wave crest (Figure 11).

In [ ]:
g.run_command('r.to.vect', 
                  input='swc_thin_long', 
                  output='swc_thin_long_v',
                  type='line',
                  overwrite=True)

In [ ]:
makefigure(['swc_thin_long', 
            'swc_thin_long_v'], 
           caption=[11, 'SWC vectorization.'])

###  SWC topological cleaning
Line features are cleaned by removing any dangle (Figure 12).

In [ ]:
g.run_command('v.generalize', 
                  input='swc_thin_long_v', 
                  output='swc_thin_long_smooth_v',
                  method='douglas',
                  threshold=2,
                  overwrite=True)

In [ ]:
makefigure(['swc_thin_long_v', 
            'swc_thin_long_smooth_v'], 
           caption=[12, 'SWC topological cleaning.'])

###  SWC smoothing
A low pass filter is used to obtain a vectorized Sand Wave Crest (Figure 13).

In [ ]:
g.run_command('v.clean', 
                  input='swc_thin_long_smooth_v', 
                  output='swc_thin_long_smooth_clean_v',
                  type='line',
                  tool='rmdangle,rmdangle,rmdangle,rmdangle',
                  threshold='5,10,20,30',
                  overwrite=True)

In [ ]:
makefigure(['swc_thin_long_smooth_v', 
            'swc_thin_long_smooth_clean_v'], 
           caption=[13, 'SWC smoothing.'])

## Identify areas covered by large scale bedforms

###  Sand wave (SW) extraction
From the TFE results (r.geomorphon), the entire landform is extracted by reclassifying the cells with feature type equal to summit, ridge, spur, and slope (feature type: class 2, 3, 5, 6) into a new raster feature with category value 1, and setting the remaining cells to null (Figure 14).

In [ ]:
g.read_command('g.region', 
                   n=4546380, 
                   s=4546202,
                   w=506185,
                   e=506571,
                   flags='ap', htable=True, sep=':')

In [ ]:
g.run_command('r.geomorphon', 
                  elevation='bathy_2015', 
                  forms='geomorphometry_sw', 
                  search=30, 
                  skip=9, 
                  flat=3.7, 
                  dist=15, 
                  step=0, 
                  start=0, 
                  overwrite=True)

In [ ]:
g.run_command('r.mapcalc' , 
                  expression="sw=if(%s==3 | %s==2 | %s==6 | %s==5 , 1, null())" % ('geomorphometry_sw', 
                                                                                   'geomorphometry_sw', 
                                                                                   'geomorphometry_sw', 
                                                                                   'geomorphometry_sw'), 
                  overwrite=True)

In [ ]:
makefigure(['geomorphometry_sw', 
            'sw'], 
           caption=[14, 'Sand wave (SW) extraction.'])

### SW clumping
The SW raster map is recategorized by grouping cells that form physically discrete areas into unique categories and assign a distinct color to each raster feature (Figure 15).

In [ ]:
g.run_command('r.clump', 
                  input='sw',
                  output='sw_clump',
                  flags='d',
                  overwrite=True)

In [ ]:
makefigure(['sw', 
            'sw_clump'], 
           caption=[15, 'SW clumping.'])

###  SW filtering by area
Each raster feature is reclassified based on its area, all the features having an area smaller than a given threshold are removed (Figure 16).

In [ ]:
g.run_command('r.area', 
                  input='sw_clump',
                  output='sw_clean',
                  lesser=1000,
                  overwrite=True)

In [ ]:
makefigure(['sw_clump', 
            'sw_clean'], 
           caption=[16, 'SWC filtering by area.'])

###  SW filling
Null cells within the discrete areas are filled with the same category of the surrounding pixels (Figure 17).

In [ ]:
g.run_command('r.neighbors', 
                  input='sw_clean', 
                  output='sw_clean_fill3', 
                  method='maximum',
                  overwrite=True)

In [ ]:
makefigure(['sw_clean', 
            'sw_clean_fill3'], 
           caption=[17, 'SW filling.'])

###  SW vectorization
Conversion from raster to vector to obtain a vector feature of type polygon representing an approximate sand wave body (Figure 18).

In [ ]:
g.read_command('g.region', 
                   n=4546380, 
                   s=4546202,
                   w=506185,
                   e=506571,
                   flags='apl', htable=True, sep=':')

In [ ]:
g.run_command('r.to.vect', 
                  input='sw_clean_fill3', 
                  output='sw_clean_fill3_v', 
                  type='area',
                  overwrite=True)

In [ ]:
g.run_command('v.clean', 
                  input='sw_clean_fill3_v',
                  output='sw_clean_fill3_v_clean',
                  tool='rmarea',
                  threshold=10,
                  overwrite=True)

In [ ]:
makefigure(['sw_clean_fill3', 
            'sw_clean_fill3_v_clean'], 
           caption=[18, 'SW vectorization.'])

## Identify lee and stoss side

###  SW and SWC overlay 
The vectorized sand wave crest is overlaid on top of the polygonal area representing the sand wave body (Figure 19).

In [ ]:
makefigure(['sw_clean_fill3_v_clean', 
            'swc_thin_long_smooth_clean_v'], 
           caption=[19, 'SW and SWC overlay.'])

### SWC clipping
The portion of the sand wave crest that is not included in the sand wave body is removed (Figure 20).

In [ ]:
g.run_command('v.overlay', 
                  ainput='swc_thin_long_smooth_clean_v', 
                  atype='line', 
                  binput='sw_clean_fill3_v_clean', 
                  out='sw_ridges_v', 
                  operator='and', 
                  olayer='0,1,0',
                  overwrite=True)

In [ ]:
makefigure(['sw_clean_fill3_v_clean', 
            'sw_ridges_v'], 
           caption=[20, 'SWC clipping.'])

### SWC buffering
A polygonal area is created by buffering the sand wave crest (buffer distance equal to the DBM cell size) (Figure 21).

In [ ]:
g.run_command('v.buffer', 
                  input='sw_ridges_v', 
                  output='sw_buffer_v', 
                  distance=1, 
                  overwrite=True)

In [ ]:
makefigure(['sw_ridges_v', 
            'sw_buffer_v'], 
           caption=[21, 'SWC buffering.'])

###  SW splitting
The buffered sand wave crest is used to split the sand wave body into two parts (Figure 22).

In [ ]:
g.run_command('v.overlay', 
                  ainput='sw_clean_fill3_v_clean', 
                  binput='sw_buffer_v', 
                  operator='not', 
                  output='sw_splitted_v', 
                  overwrite=True)

In [ ]:
makefigure(['sw_clean_fill3_v_clean', 
            'sw_splitted_v'], 
           caption=[22, 'SW splitting.'])

### Identification of stoss and lee sides
The identification is achieved by performing an univariate statistical analysis on the DBM slope, using each sand wave side as a mask. The side with the higher values of the slope is assigned the label of lee side and is colored in grey, while the side with the lower values of the slope is assigned the label of stoss side (Figure 23).

In [ ]:
g.run_command('v.colors', 
                  map='sw_splitted_v',
                  use='cat',
                  color='random')

In [ ]:
makefigure(['sw_clean_fill3_v_clean', 
            'sw_splitted_v'], 
           caption=[23,'Identification of dune\'s side.'])

In [ ]:
g.run_command('v.to.rast', 
                  input='sw_splitted_v', 
                  output='sw_splitted', 
                  type='area', use='cat', overwrite=True)
                  

In [ ]:
makefigure(['sw_splitted', 
            'sw_splitted_v'], 
           caption=[18, 'SW vectorization.'])

In [ ]:
g.run_command('r.slope.aspect',
                  elevation='bathy_2015',
                  slope='bathy_2015_slope_deg',
                  overwrite=True)

In [ ]:
stats = g.read_command('r.univar', 
                  map='bathy_2015_slope_deg', 
                  zones='sw_splitted', 
                  flags='ge',
                  overwrite=True, sep='=')

In [ ]:
side_a, side_b = g.read_multifeature(stats, htable=True)

In [ ]:
side_a['category']=side_a['zone'].split('Category')[1]
side_a['zone']=side_a['zone'].split('Category')[0]
side_b['category']=side_b['zone'].split('Category')[1]
side_b['zone']=side_b['zone'].split('Category')[0]

In [ ]:
display(side_a, side_b)

In [ ]:
if side_a['mean'] > side_b['mean']:
    print(side_a['category'], 'is lee side')
    !v.db.update sw_splitted_v column=a_label value=lee where="cat={side_a['category']}"
    !v.db.update sw_splitted_v column=a_label value=stoss where="cat={side_b['category']}"
else: 
    print(side_b['category'], 'is lee side')
    !v.db.update sw_splitted_v column=a_label value=lee where="cat={side_b['category']}"
    !v.db.update sw_splitted_v column=a_label value=stoss where="cat={side_a['category']}"

In [ ]:
grass.read_command('v.db.select', map='sw_splitted_v').decode().strip().split('\n')

## Compute sand wave’s metrics

### SW Length

In [ ]:
grass.run_command('v.build.polylines',
                  type='line',
                  input='sw_ridges_v',
                  output='sw_ridges_v_poliline',
                  cats='no', 
                  overwrite=True)

In [ ]:
grass.run_command('v.category', 
                  option='add',
                  type='line',
                  input='sw_ridges_v_poliline',
                  output='sw_ridges_v_polilineCAT', 
                  overwrite=True)

In [ ]:
grass.run_command('v.db.addtable',
                  map='sw_ridges_v_polilineCAT',
                  layer=1,
                  columns='Length double precision')

In [ ]:
grass.run_command('v.to.db',
                  type='line',
                  layer=1,
                  map='sw_ridges_v_polilineCAT',
                  option='cat',
                  columns='cat')

In [ ]:
grass.run_command('v.to.db',
                  type='line',
                  layer=1,
                  units='meters',
                  map='sw_ridges_v_polilineCAT',
                  option='length',
                  columns='Length')

In [ ]:
g.read_command('v.db.select',
                   map='sw_ridges_v_polilineCAT', 
                   separator=',', sep='\n')

### SWC 3D length

In [ ]:
grass.run_command('v.split',
                  input='sw_ridges_v_polilineCAT',
                  output='sw_ridges_v_polilineCAT_split',
                  length=1,
                  flags='f', 
                  overwrite=True)

In [ ]:
grass.run_command('v.to.points',
                  input='sw_ridges_v_polilineCAT_split',
                  output='sw_ridges_v_polilineCAT_splitP',
                  dmax=1, 
                  overwrite=True)

In [ ]:
grass.run_command('v.db.droptable', 
                  flags='f', 
                  map='sw_ridges_v_polilineCAT_splitP',
                  layer=1)

In [ ]:
grass.run_command('v.db.droptable', 
                  flags='f', 
                  map='sw_ridges_v_polilineCAT_splitP',
                  layer=2)

In [ ]:
grass.run_command('v.db.addtable', 
                  map='sw_ridges_v_polilineCAT_splitP', 
                  layer=2, 
                  columns='x DOUBLE PRECISION, y DOUBLE PRECISION, z DOUBLE PRECISION')


In [ ]:
grass.run_command('v.to.3d',
                  input='sw_ridges_v_polilineCAT_splitP',
                  output='sw_ridges_v_polilineCAT_splitP3D',
                  type='point', 
                  column='z', 
                  layer=2,
                  overwrite=True)


In [ ]:
grass.run_command('v.drape', 
                  input='sw_ridges_v_polilineCAT_splitP3D', 
                  output='sw_ridges_v_polilineCAT_splitP3D_draped', 
                  elevation='bathy_2015',
                  overwrite=True)

In [ ]:
grass.run_command('v.to.db', 
                  type='point', 
                  layer=2, 
                  map='sw_ridges_v_polilineCAT_splitP3D_draped', 
                  option='coor', 
                  columns='x,y,z')


In [ ]:
dp = g.read_command('v.db.select',
                    map='sw_ridges_v_polilineCAT_splitP3D_draped', 
                    layer=2, 
                    separator=',', 
                    std=True)

In [ ]:
import numpy as np
import scipy.spatial

In [ ]:
matx = np.array([float(i.split(',')[1]) for i in dp[1:]])
maty = np.array([float(i.split(',')[2]) for i in dp[1:]])
matz = np.array([float(i.split(',')[3]) for i in dp[1:]])

In [ ]:
pp = np.array([list(i) for i in zip(matx, maty, matz)])

In [ ]:
dist = scipy.spatial.distance.cdist(pp, pp)

#### Distance between endpoints

In [ ]:
dist.max()

#### 3D Distance along the line

In [ ]:
dist.diagonal(1).sum()

### SW height
Derivation of SW height by generating a series of vertical profiles along several transects perpendicular to SWC. The spacing between the equidistant vertical profiles is given by an input parameter and set to 10 m as default value, the length of each profile is also variable by the user (by default is set to be the same length of the sand wave ridge) (Figure 24).

In [ ]:
grass.run_command('v.transects', 
                  input='sw_ridges_v', 
                  output='sw_ridges_transect_v', 
                  transect_spacing=1, 
                  dleft=70, 
                  dright=70, 
                  overwrite=True)

In [ ]:
makefigure(['sw_splitted_v', 
            'sw_ridges_transect_v'], 
           caption=[24, 'Generate transects to be used as vertical profiles.'])


In [ ]:
%matplotlib inline


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
lines = grass.read_command('v.out.ascii', 
                           input='sw_ridges_transect_v', 
                           output='-', 
                           type='line', 
                           format='standard', 
                           overwrite=True).decode().strip().split('\n')

In [ ]:
lines = g.read_command('v.out.ascii', 
                           input='sw_ridges_transect_v', 
                           output='-', 
                           type='line', 
                           format='standard', 
                           overwrite=True, sep='\n')

In [ ]:
coords = getcoords(lines)

In [ ]:
cc = [{'layer': 'bathy_2015', 
       'resolution': 1,
       'coordinates': v} for i,v in coords.items()]

In [ ]:
from multiprocessing import Pool

prof = []
with Pool(8) as p:
    prof.append(p.map(r.makeprofiles, cc))

pr = []
with Pool(8) as p:
    pr.append(p.map(getprofile, prof[0]))

In [ ]:
plt.figure(figsize=(12,4))
plt.grid(True)
for i in pr[0]:
    plt.plot(i[:,2], i[:,3])

#### Height

In [ ]:
dz = [float(np.abs(np.min(i[:,3])) - np.abs(np.max(i[:,3]))) for i in pr[0]]
H = max(dz)
H

#### Vertical Profile

In [ ]:
zc = pr[0][np.argmax(dz)][:,3]
dc = pr[0][np.argmax(dz)][:,2]

In [ ]:
plt.figure(figsize=(12,4))
plt.grid(True)
plt.plot(dc,zc,'-');
plt.ylabel('Depth [m]')
plt.xlabel('Distance [m]');

#### Summit

In [ ]:
x = pr[0][np.argmax(dz)][np.argmax(pr[0][np.argmax(dz)][:,3]),0]
y = pr[0][np.argmax(dz)][np.argmax(pr[0][np.argmax(dz)][:,3]),1]
z = pr[0][np.argmax(dz)][np.argmax(pr[0][np.argmax(dz)][:,3]),3]
h = max(dz)

In [ ]:
x,y,z,h

### SW Width


In [ ]:
W = h*np.tan(float(side_a['mean'])) + h*np.tan(float(side_b['mean']))

In [ ]:
W

# SWC Validation

## SWC reconstruction and matching

In [ ]:
xc = [i[np.argmax(i[:,3]), 0] for i in pr[0]]
yc = [i[np.argmax(i[:,3]), 1] for i in pr[0]]
zc = [i[np.argmax(i[:,3]), 3] for i in pr[0]]

In [ ]:
plt.figure(figsize=(12,4))
plt.grid(True)
plt.plot(xc,yc,'o')
plt.plot(x,y,'or');

In [ ]:
xyz = [i for i in map(lambda x: str(x).replace('(','').replace(')','').replace(' ',''), zip(xc,yc,zc))]

In [ ]:
with open('tmp/t.csv', 'w') as t:
    for i in xyz:
        t.write(i)
        t.write('\n')

In [ ]:
grass.run_command('v.in.ascii', 
                  input='tmp/t.csv', 
                  output='crest_point', 
                  separator=',', 
                  format='point', 
                  columns="x double precision, y double precision, z double precision",
                  z=3,
                  flags='z', 
                  overwrite=True)

In [ ]:
makefigure(['sw_splitted_v', 
            'crest_point'], 
           caption=[24, 'Generate transects to be used as vertical profiles.'])

In [ ]:
grass.run_command('v.db.addtable', 
                  map='crest_point', 
                  table='crest_point', 
                  columns="dist double precision, dist2 double precision", 
                  overwrite=True)

In [ ]:
grass.run_command('v.db.connect', 
                  map='crest_point', 
                  table='crest_point', 
                  flags='p',
                  overwrite=True)

In [ ]:
#grass.run_command('v.distance', from='crest_point', to='sw_ridges_v', upload='dist', column='dist')

In [ ]:
#grass.run_command('v.distance',
#                  from='crest_point',
#                  to='swc_thin_long_v',
#                  upload='dist',
#                  column='dist2')

In [ ]:
!v.distance from=crest_point to=sw_ridges_v upload=dist column=dist --qq

In [ ]:
!v.distance from=crest_point to=swc_thin_long_v upload=dist column=dist2 --qq

In [ ]:
dist = g.read_command('v.db.select', 
                          map='crest_point', std=True)

In [ ]:
distance1 = [i.split("|")[1] for i in dist[1:]]
distance2 = [i.split("|")[2] for i in dist[1:]]

#### Mean displacements

In [ ]:
np.array(distance1, np.dtype(float)).mean(), np.array(distance1, np.dtype(float)).std() 

In [ ]:
np.array(distance2, np.dtype(float)).mean(), np.array(distance2, np.dtype(float)).std()

Load custom ccs (not run)
```python
from IPython.core.display import HTML


def css_styling():
    styles = open("custom.css", "r").read()
    return HTML(styles)
css_styling()
``` 

In [ ]:
end=dt.datetime.now()
elapsed=end-start
elapsed.seconds
